In [ ]:
library(crayon, quietly=T); # load first to avoid error relating to unloading this package
install.packages("nnet", quiet=T)      # 3 sec. multinomial logistic regression: multinom()
install.packages("MASS", quiet=T)      # 9 sec. binary logistic regression:  glm(..., family=binomial);   LDA/QDA: lda(), qda()
install.packages("tidyverse", quiet=T) # 7 sec.
install.packages("glmnet", quiet=T)    # 1 min. multinomial logistic regression: glmnet(..., family=multinomial).
install.packages("FNN", quiet=T)       # KNN Regression. Docs: https://cran.r-project.org/web/packages/FNN/FNN.pdf
install.packages('reshape2', quiet=T)  # has an easy melt() function (vs gather, stack, reshape)
devtools::install_github("mkearney/kaggler", quiet=T) # https://github.com/mkearney/kaggler
library(httr, quietly=T); library(stringr, quietly=T); library(glmnet, quietly=T); library(nnet, quietly=T); library(MASS, quietly=T); 
library(tidyverse, quietly=T); library(reshape2, quietly=T); library(RColorBrewer, quietly=T);
options(repr.plot.width=20, repr.plot.height=8)   # change plot size

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘Rcpp’, ‘RcppEigen’


also installing the dependency ‘plyr’


Installing 3 packages: utf8, openssl, jsonlite

Loaded glmnet 4.1-6

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.3.0      ✔ forcats 0.5.2 
✔ readr   2.1.3      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::%+%()  masks crayon::%+%()
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ dplyr::select() masks MASS::select()
✖ tidyr::unpack() masks Matrix::unpack()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [ ]:
Competition = '6feb23shop'
FileCSV = 'XY_Shop.csv'
FileZip = 'XY_Shop.zip'
# capture.output( kaggler::kgl_auth(creds_file='kaggle.json'), file='log_auth.txt')     # log is deleted when session ends
url = paste0('https://www.kaggle.com/api/v1/competitions/data/download/', Competition,'/', FileCSV)
FileURL = httr::GET(url, kaggler::kgl_auth())$url   # extract full path to zipped data file
download.file(FileURL, FileZip, quiet=T, mode="wb") # download zipped data file
system(paste0('unzip ', FileZip), intern=T)         # oddly unzip() fails to properly unzip
kaggler::kgl_competitions_leaderboard_view(Competition)
timeLimit = 2*60  # seconds

[1] "Archive:  XY_Shop.zip"                
[2] "  inflating: XY_Shop.csv             "

,teamNameNullable,scoreNullable,teamId,teamName,hasTeamName,submissionDate,score,hasScore
,<chr>,<chr>,<int>,<chr>,<lgl>,<chr>,<chr>,<lgl>
1,,0.80392,9841595,,TRUE,2022-02-21T04:44:25.9633333Z,0.80392,TRUE
2,,0.79960,9841594,,TRUE,2022-02-21T03:28:25.3433333Z,0.79960,TRUE


In [ ]:
XY = read.csv(FileCSV, na.strings="", stringsAsFactors=T)  # replace "" with NA
YCol = 'Rev'       # target column
NumericCols =  unlist(lapply(XY, is.numeric))   # find numeric columns
vX = XY[is.na(XY[YCol]), !(names(XY) %in% c(YCol))]  # testing inputs only
tXY = XY[!is.na(XY[YCol]), ]  # training inputs with output
tY = tXY[YCol]                # training response
tX = tail(tXY[,1:(ncol(tXY)-1)])
tail(vX, 2)  # a few rows from the test set
tail(tX, 2)  # a few rows from the train set
tail(tXY, 2) # a few rows from the training set

,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
49999,4,357.05,0,0,27,1278.00,0.00000,0.006812,14.53818,0,11,2,2,5,2,0,0
50000,0,0.00,0,0,26,1646.42,0.00755,0.032493,0.00000,0,2,2,2,4,3,0,1


,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
499999,0,0.00,0,0,27,1185.14,0,0.001593,0,0,5,2,2,2,3,0,1
500000,6,51.36,0,0,59,1898.21,0,0.003224,0,0,12,2,2,2,1,0,0


,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
499999,0,0.00,0,0,27,1185.14,0,0.001593,0,0,5,2,2,2,3,0,1,0
500000,6,51.36,0,0,59,1898.21,0,0.003224,0,0,12,2,2,2,1,0,0,0


In [ ]:
t0 = Sys.time()

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
set.seed(0)   # always seed your experiments for reproducibility
m = multinom('Rev ~ . ', data=tXY[1:10000,], MaxNWts=10000)

# weights:  19 (18 variable)
initial  value 6931.471806 
iter  10 value 4961.223494
iter  20 value 4306.653583
final  value 4304.931127 
converged


In [ ]:
t(head(pY <- setNames(data.frame(predict(m, vX)), YCol), 20))  # make predictions on the test inputs, vX

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
write.csv(cbind(id=1:nrow(pY), pY), file='MySubmission.csv', row.names=F)  # write to CSV file

In [ ]:
table(tY$Rev)/nrow(tY)
table(pY$Rev)/nrow(pY)


        0         1 
0.7758667 0.2241333 


      0       1 
0.92058 0.07942 

# **References:**

1. Remember to cite your sources!


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
RunTime = round(difftime(Sys.time(), t0, units = "secs")[[1]])
msg = paste0('Runtime is ',RunTime,' sec. Time limit is ', timeLimit, ' sec')
cat(bold(ifelse(timeLimit<RunTime, red(paste0('Limit exceeded!!! ', msg)), green(msg))))

Runtime is 1 sec. Time limit is 120 sec

## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)